In [6]:
import panel as pn
import pandas as pd
import hvplot.pandas

from bokeh.sampledata import stocks

pn.extension(template='fast-list')
stocks

<module 'bokeh.sampledata.stocks' from 'C:\\Users\\mattr\\anaconda3\\envs\\pyvizenv\\lib\\site-packages\\bokeh\\sampledata\\stocks.py'>

This example is meant to make it easy to compare and contrast the different APIs Panel provides to declare apps and dashboards. Specifically, it compares four different implementations of the same app using 1) the quick and easy ``interact`` function, 2) more flexible reactive functions, 3) declarative Param-based code, and 4) explicit callbacks.

Before comparing the different approaches, we will first declare some components of the app that will be shared, including the title of the app, a set of stock tickers, a function to return a dataframe given the stock ``ticker`` and the rolling mean ``window_size``, and another function to return a plot given those same inputs:

In [9]:
title = '## Stock Explorer hvPlot'

tickers = ['AAPL', 'FB', 'GOOG', 'IBM', 'MSFT']

def get_df(ticker, window_size):
    df = pd.DataFrame(getattr(stocks, ticker))
    print(df)
    df['date'] = pd.to_datetime(df.date)
    print(df)
    return df.set_index('date').rolling(window=window_size).mean().reset_index()

def get_plot(ticker, window_size):
    df = get_df(ticker, window_size)
    return df.hvplot.line('date', 'close', grid=True, responsive=True, height=300)

### Interact

In the ``interact`` model the widgets are automatically generated from the arguments to the function or by providing additional hints to the ``interact`` call. This is a very convenient way to generate a simple app, particularly when first exploring some data.  However, because widgets are created implicitly based on introspecting the code, it is difficult to see how to modify the behavior.  Also, to compose the different components in a custom way it is necessary to unpack the layout returned by the ``interact`` call, as we do here:

In [10]:
interact = pn.interact(get_plot, ticker=tickers, window_size=(1, 21, 5))

pn.Row(
    pn.Column(title, interact[0]),
    interact[1]
)

            date    open    high     low   close    volume  adj_close
0     2000-03-01  118.56  132.06  118.50  130.31  38478000      31.68
1     2000-03-02  127.00  127.94  120.69  122.00  11136800      29.66
2     2000-03-03  124.87  128.23  120.00  128.00  11565200      31.12
3     2000-03-06  126.00  129.13  125.00  125.69   7520000      30.56
4     2000-03-07  126.44  127.44  121.12  122.87   9767600      29.87
...          ...     ...     ...     ...     ...       ...        ...
3265  2013-02-25  453.85  455.12  442.57  442.80  13306400     437.00
3266  2013-02-26  443.82  451.54  437.66  448.97  17910700     443.09
3267  2013-02-27  448.43  452.44  440.65  444.57  20976800     438.75
3268  2013-02-28  444.05  447.87  441.40  441.40  11518400     435.62
3269  2013-03-01  438.00  438.18  429.98  430.47  19730300     424.83

[3270 rows x 7 columns]
           date    open    high     low   close    volume  adj_close
0    2000-03-01  118.56  132.06  118.50  130.31  38478000      31.

Row
    [0] Column
        [0] Markdown(str)
        [1] Column
            [0] Select(name='ticker', options=['AAPL', 'FB', ...], value='AAPL')
            [1] IntSlider(end=21, name='window_size', start=1, step=5, value=11)
    [1] Row
        [0] HoloViews(Curve, name='interactive02327')

### Reactive

The reactive programming model is similar to the ``interact`` function but relies on the user (a) explicitly instantiating widgets, (b) declaring how those widgets relate to the function arguments (using the ``bind`` function), and (c) laying out the widgets and other components explicitly. In principle we could reuse the ``get_plot`` function from above here but for clarity we will repeat it:

In [5]:

window = pn.widgets.IntSlider(name='Window Size', value=6, start=1, end=21)
# tickers = input('Ticker')
df = pd.DataFrame()
ticker = pn.widgets.input.TextInput()
ticker.value = 'AAPL'
def get_plot(ticker, window_size):
    df = get_df(ticker, window_size)
    return df.hvplot.line('date', 'close', grid=True, responsive=True, height=300)

pn.Row(
    pn.Column(title, ticker, window),
    pn.bind(get_plot, ticker, window)
)

Row
    [0] Column
        [0] Markdown(str)
        [1] TextInput(value='AAPL')
        [2] IntSlider(end=21, name='Window Size', start=1, value=6)
    [1] ParamFunction(function)

### Parameterized class

Another approach expresses the app entirely as a single ``Parameterized`` class with parameters to declare the inputs, rather than explicit widgets. The parameters are independent of any GUI code, which can be important for maintaining large codebases, with parameters and functionality defined separately from any GUI or panel code. Once again the ``depends`` decorator is used to express the dependencies, but in this case the dependencies are expressed as strings referencing class parameters, not parameters of widgets. The parameters and the ``plot`` method can then be laid out independently, with Panel used only for this very last step.

In [5]:
import param

class StockExplorer(param.Parameterized):
    
    ticker = param.Selector(default='AAPL', objects=tickers)  
    window_size = param.Integer(default=6, bounds=(1, 21))
    
    @param.depends('ticker', 'window_size')
    def plot(self):
        return get_plot(self.ticker, self.window_size)
    
explorer = StockExplorer(name='Stock Explorer hvPlot')
pn.Row(
    pn.Column(explorer.param),
    explorer.plot
)

Row
    [0] Column
        [0] Column(margin=5, name='Stock Explorer hvPlot', width=300)
            [0] StaticText(value='<b>Stock Explorer h...)
            [1] Select(name='Ticker', options=OrderedDict([('AAPL', ...]), value='AAPL')
            [2] IntSlider(end=21, name='Window size', start=1, value=6)
    [1] ParamMethod(method)

### Callbacks

The above approaches are all reactive in some way, triggering actions whenever manipulating a widget causes a parameter to change, without users writing code to trigger callbacks explicitly. Explicit callbacks allow complete low-level control of precisely how the different components of the app are updated, but they can quickly become unmaintainable because the complexity increases dramatically as more callbacks are added. The approach works by defining callbacks using the ``.param.watch`` API that either update or replace the already rendered components when a watched parameter changes:

In [6]:
ticker = pn.widgets.Select(name='Ticker', options=['AAPL', 'FB', 'GOOG', 'IBM', 'MSFT'])
window = pn.widgets.IntSlider(name='Window', value=6, start=1, end=21)

row = pn.Row(
    pn.Column(title, ticker, window),
    get_plot(ticker.options[0], window.value)
)

def update(event):
    row[1].object = get_plot(ticker.value, window.value)

ticker.param.watch(update, 'value')
window.param.watch(update, 'value')

row

Row
    [0] Column
        [0] Markdown(str)
        [1] Select(name='Ticker', options=['AAPL', 'FB', ...], value='AAPL')
        [2] IntSlider(end=21, name='Window', start=1, value=6)
    [1] HoloViews(Curve)

In practice, different projects will be suited to one or the other of these APIs, and most of Panel's functionality should be available from any API.

## App

This notebook may also be served as a standalone application by running it with `panel serve stocks_hvplot.ipynb`. Above we enabled a custom `template`, in this section we will add components to the template with the `.servable` method:

In [ ]:
ticker.servable(area='sidebar')
window.servable(area='sidebar')

pn.panel("""This example compares **four different implementations of the same app** using 

- the quick and easy ``interact`` function, 
- more flexible *reactive* functions,
- declarative *Param-based* code, and 
- explicit *callbacks*.""").servable()

pn.panel(pn.bind(get_plot, ticker, window)).servable(title='hvPlot Stock Explorer');